# Demo for AIFlow class


Here is a list of all methods from the provided `aiflow.py` file along with a one-line description for each:

1. **chroma_query_result_to_text(obj)**: Converts a query result to a string for use in the class.

### AIFlow Class Methods

#### Initialization and Configuration

2. **__init__(self, api_key, model="gpt-4", temperature=0, max_tokens=150)**: Initializes the AIFlow class with API key and model parameters.
3. **set_temperature(self, temperature=0)**: Sets the temperature for the model.
4. **set_model(self, model="gpt-4")**: Sets the model to be used.
5. **set_max_tokens(self, max_tokens=150)**: Sets the maximum number of tokens for the model.
6. **set_json_output(self, json_mode=False)**: Sets the output mode to JSON.
7. **show_model_config(self)**: Prints the current model configuration.
8. **set_output_folder(self, folder="")**: Sets the default folder for output files.

#### Debugging and Data Management

9. **show_self_data(self)**: Prints internal data such as chat messages and context maps.
10. **clear_self_data(self)**: Clears all internal data.

#### Chat Methods

11. **pretty_print_messages(self)**: Prints chat messages in a readable format.
12. **pretty_print_messages_to_file(self, file_name="output.txt", html=True)**: Saves chat messages to a file and optionally returns an HTML link for download.
13. **set_system_prompt(self, prompt="")**: Sets the system prompt for the chat.
14. **add_user_chat(self, prompt, label="latest")**: Adds a user chat message and gets a response from the model.
15. **filter_messages(self, func)**: Filters chat messages based on a provided function.
16. **reduce_messages_to_text(self, func)**: Reduces chat messages to text using a provided function.

#### Completion Methods

17. **completion(self, prompt, label="latest")**: Generates a completion for the given prompt.

#### OpenAI API Interaction

18. **call_openai_chat_api(self, messages=[])**: Calls the OpenAI chat API with provided messages.

#### Context Management

19. **replace_tags_with_content(self, input_string="")**: Replaces tags in a string with corresponding context content.
20. **copy_latest_to(self, label="latest")**: Copies the latest context to a specified label.
21. **transform_context(self, label="latest", func=lambda x: x)**: Transforms context using a provided function.
22. **set_context_of(self, content="", label="latest")**: Sets the content for a specified context label.
23. **delete_context(self, label="latest")**: Deletes a specified context label.
24. **show_context_of(self, label="latest")**: Prints the content of a specified context label.
25. **show_context_keys(self)**: Prints all context keys.
26. **read_textfile_to_context(self, filename, label="latest_file")**: Reads a text file into context.
27. **dump_context_to_files(self)**: Dumps all context data to text files.
28. **dump_context_to_markdown(self, output_filename="content.md")**: Dumps context data to a Markdown file.
29. **generate_heading_for_context(self, label="latest", prompt="Generate a short 10 word summary of the following content:\n")**: Generates a heading for context content.
30. **dump_context_to_docx(self, output_filename)**: Dumps context data to a DOCX file.

#### Chat-in-chat Support (not return self, but returning the last output)

31. **return_latest_to_text(self)**: Returns the latest context as text.
32. **return_context_to_text(self, label="latest")**: Returns the context of a specified label as text.
33. **return_reduce_messages_to_text(self, func)**: Returns reduced chat messages as text using a provided function.

#### State Management

34. **save_state(self, filename="state.json")**: Saves the current state to a JSON file.
35. **load_state(self, filename="state.json")**: Loads state from a JSON file.

#### Image Generation and Handling

36. **generate_image(self, model="dall-e-2", style="vivid", response_format="url", prompt="A white siamese cat", size="1024x1024", quality="standard", n=1, label="latest_image", html=False)**: Generates an image based on a prompt.
37. **dump_image_to_file(self, label="latest_image", filename="")**: Saves an image to a file.

#### Vision Model

38. **interpret_image(self, image="", prompt="What's in this image?", model="gpt-4o", label="latest", detail="low", max_tokens=300)**: Interprets an image using a vision model.

#### Audio Generation and Handling

39. **generate_speech(self, model="tts-1", voice="alloy", response_format="mp3", prompt="A white siamese cat", speed=1, filename="", label="latest_speech", html=False)**: Generates speech from a prompt.
40. **generate_transcription(self, filename="", model="whisper-1", language="en", prompt="", response_format="text", temperature=0, label="latest")**: Transcribes audio from a file.

#### Moderation

41. **generate_moderation(self, prompt="", label="latest_moderation")**: Generates moderation output for a given prompt.

## Magic
There is magic in the class - everytime you do a text prompt, the last result is stored in the context map under `latest`. So you can use that one as context variable for further processing or reference in queries.

## The demo

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_KEY")
from aiflow import AIFlow


# Just an empty flow

In [2]:
flow1 = AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=0)
(
    flow1
    .show_model_config()
    .show_self_data()
    .pretty_print_messages()
    .pretty_print_messages_to_file(file_name="nothing.txt")
)


Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 0
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}
Tell a joke

'CompletionUsage' object has no attribute 'total_token'



# Chat demo

In [3]:
chatbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
chatbot.show_model_config().show_self_data()

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


In [6]:
(    chatbot
        .show_model_config()
        .set_system_prompt("You are a helpful cooking assistant always eager to give a recipe for whatever the user says.")
        .add_user_chat("How is the weather here?")
        .show_self_data()
        .get_token_usage()
)

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
You are a helpful cooking assistant always eager to give a recipe for whatever the user says.

How is the weather here?
I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either.

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a 

In [7]:
chatbot.add_user_chat("Ok. Give me a recipe inspired by weather")

Ok. Give me a recipe inspired by weather
Sure! How about a refreshing summer salad inspired by sunny and warm weather?

Sunny Summer Salad Recipe:

Ingredients:
- 1 large ripe mango, diced
- 1 cup cherry tomatoes, halved
- 1 cucumber, diced
- 1/4 red onion, thinly sliced
- 1/4 cup fresh basil, chopped
- Juice of 1 lemon
- 2 tbsp extra virgin olive oil
- Salt and pepper, to taste

Instructions:
1. In a large bowl, combine the diced mango, cherry tomatoes, cucumber, red onion, and basil.
2. In a small bowl, whisk together the lemon juice and olive oil. Pour over the salad and toss to combine.
3. Season with salt and pepper to taste.
4. Chill in the refrigerator for at least 30 minutes before serving to allow the flavors to meld together.
5. Serve as a side dish or enjoy on its own for a light and vibrant meal perfect for a sunny summer day.

Enjoy your sunny summer salad!



In [8]:
chatbot.latest_to_context("dish")


In [9]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh 

## Message manipulations

In [10]:
chatbot.reduce_messages_to_text(lambda x: (print(x), "Replacing latest with all sorts of nothing")[1]).show_self_data()

[{'role': 'system', 'content': 'You are a helpful cooking assistant always eager to give a recipe for whatever the user says.'}, {'role': 'user', 'content': 'How is the weather here?'}, {'role': 'assistant', 'content': "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."}, {'role': 'user', 'content': 'Ok. Give me a recipe inspired by weather'}, {'role': 'assistant', 'content': "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and chop all the fruits into bite-sized pieces.\n2. In a large bowl, combine the fruits and 

In [11]:
chatbot.filter_messages(lambda items: [{**item, "content": "You are an artist very good in poetry"} if item.get("role") == "system" else item for item in items])

In [12]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

## Just completions - no chat

In [4]:
chatbot.completion("Give me 10 random names")

Give me 10 random names

1. Jasmine
2. Liam
3. Sofia
4. Noah
5. Isabella
6. Ethan
7. Zoe
8. Mason
9. Ava
10. Lucas



In [14]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

In [3]:
chatbot.completion("Give the following names in reverse order: [latest].")

Give the following names in reverse order: [latest].

[latest] --> stetila



## Saving and loading state
To avoid having the rerun all queries when you restart the kernel etc..

In [4]:
chatbot.save_state(filename="mystate.json")


In [6]:
chatbot.load_state(filename="mystate.json").show_self_data()

Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


## Some context manipulations

In [5]:
chatbot.show_self_data()

Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


In [19]:
chatbot.set_context(content="Nothing", label="stuff").show_self_data().delete_context(label="stuff").show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

## Dumping content to files

In [20]:
chatbot.dump_context_to_files().dump_context_to_markdown()

In [9]:
chatbot.generate_heading_for_context(label="dish")


Generate a short oneline summary of the following content:
Sure! How about a refreshing summer salad inspired by sunny and warm weather?

Sunny Summer Salad Recipe:

Ingredients:
- 1 large ripe mango, diced
- 1 cup cherry tomatoes, halved
- 1 cucumber, diced
- 1/4 red onion, thinly sliced
- 1/4 cup fresh basil, chopped
- Juice of 1 lemon
- 2 tbsp extra virgin olive oil
- Salt and pepper, to taste

Instructions:
1. In a large bowl, combine the diced mango, cherry tomatoes, cucumber, red onion, and basil.
2. In a small bowl, whisk together the lemon juice and olive oil. Pour over the salad and toss to combine.
3. Season with salt and pepper to taste.
4. Chill in the refrigerator for at least 30 minutes before serving to allow the flavors to meld together.
5. Serve as a side dish or enjoy on its own for a light and vibrant meal perfect for a sunny summer day.

Enjoy your sunny summer salad!

Delicious and refreshing summer salad recipe with mango, cherry tomatoes, cucumber, basil, and lem

In [ ]:
chatbot.dump_context_to_docx(output_filename="output.docx")

In [10]:
chatbot.show_self_data().get_token_usage()

Chat Messages:
[
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "Sure! How about a refreshing summer salad inspired by sunny and warm weather?\n\nSunny Summer Salad Recipe:\n\nIngredients:\n- 1 large ripe mango, diced\n- 1 cup cherry tomatoes, halved\n- 1 cucumber, diced\n- 1/4 red onion, thinly sliced\n- 1/4 cup fresh basil, chopped\n- Juice of 1 lemon\n- 2 tbsp extra virgin olive oil\n- Salt and pepper, to taste\n\nInstructions:\n1. In a large bowl, combine the diced mango, cherry tomatoes, cucumber, red onion, and basil.\n2. In a small bowl, whisk together the lemon juice and olive oil. Pour over the salad and toss to combine.\n3. Season with salt and pepper to taste.\n4. Chill in the refrigerator for at least 30 minutes before serving to allow the flavors to meld together.\n5. Serve as a side dish or enjoy on its own for a light and vibrant meal perfect for a sunny summer day.\

TypeError: Object of type AIFlow is not JSON serializable

## Data loading

In [21]:
chatbot.read_textfile_to_context(filename="output.txt").show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

# Combining AIFlows

In [22]:
otherbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
otherbot.completion("Let's say you are a random person hungry for food. What would you like to eat?")

Let's say you are a random person hungry for food. What would you like to eat?

As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.



In [23]:
otherbot.return_latest_to_text()

"As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings."

In [24]:
sillybot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
sillybot.set_context(content=otherbot.return_latest_to_text(), label="food_choice").show_self_data()
sillybot.completion(prompt="Give a silly comment on the following choice of food: [food_choice]").show_self_data()

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings."
}

Images Map:
{}

Audio Map:
{}
Give a silly comment on the following choice of food: [food_choice]

I don't have taste buds, but I'm pretty sure all food tastes better when eaten in pajamas.

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.",
    "latest_prompt": "Give a silly comment on the following choice of food: As a language model AI, I don't have the ability to eat, but some popular food options for people

# Using vectordatabase Chroma

In [25]:
# https://docs.trychroma.com/guides
import chromadb
chroma_client = chromadb.Client() # chromadb.PersistentClient(path="/path/to/save/to")
collection = chroma_client.create_collection(name="my_collection")

In [26]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [27]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['id1']], 'distances': [[1.0404009819030762]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['This is a document about pineapple']], 'uris': None, 'data': None}


In [28]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection

2

In [29]:
from aiflow import chroma_query_result_to_text
print(chroma_query_result_to_text(results))

This is a document about pineapple


In [30]:
sillybot.set_context(content=chroma_query_result_to_text(results), label="found_doc")
sillybot.completion("Say something really silly about this random text: [found_doc]").show_self_data()

Say something really silly about this random text: [found_doc]

Did you know that pineapples are actually tiny alien spaceships in disguise? Watch out for flying fruit in your grocery store!

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.",
    "latest_prompt": "Say something really silly about this random text: This is a document about pineapple",
    "latest": "Did you know that pineapples are actually tiny alien spaceships in disguise? Watch out for flying fruit in your grocery store!",
    "found_doc": "This is a document about pineapple"
}

Images Map:
{}

Audio Map:
{}


# Working with media

https://platform.openai.com/docs/api-reference/images/create

In [7]:
flow1.generate_image(
    size="256x256", prompt="A big house in the desert.", html=False
).show_self_data().dump_image_to_file(filename="generated_output/an_image.jpg")

flow1.generate_image(size="256x256", prompt="A tomato plant on a car.", html=True)

Chat Messages:
[]

Context Map:
{
    "latest_image_prompt": "A big house in the desert.",
    "latest_revised_image_prompt": null
}

Images Map:
{
    "latest_image": "https://oaidalleapiprodscus.blob.core.windows.net/private/org-qiwjaBjm9TZ58jKWInrRESeh/user-krnk9YsqIV2WoGdKtr7UH5kB/img-znRduusEimp0JVPNfnYo8B9a.png?st=2024-06-22T12%3A26%3A55Z&se=2024-06-22T14%3A26%3A55Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-06-21T23%3A47%3A27Z&ske=2024-06-22T23%3A47%3A27Z&sks=b&skv=2023-11-03&sig=xwSF86vXpvpZ8sJe06E/gynCVYjhZycBwtu3eEmxBsg%3D"
}

Audio Map:
{}
Saving  latest_image generated_output/an_image.jpg


https://platform.openai.com/docs/guides/vision

In [8]:
flow1.interpret_image(
    image="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
)

AttributeError: 'AIFlow' object has no attribute 'interpret_image'

https://platform.openai.com/docs/api-reference/audio/createSpeech

In [9]:
flow1.generate_speech(prompt="Jump for joy!", filename="generated_output/jump.mp3")

https://platform.openai.com/docs/api-reference/audio/createTranscription

In [11]:
flow1.get_token_usage().generate_transcription(filename="generated_output/jump.mp3").get_token_usage()

Jump for joy.



https://platform.openai.com/docs/guides/moderation/overview

In [12]:
flow1.generate_moderation("This is stupid!")

{
  "id": "modr-9cv6vvgQpwnjImjZ91kuAaUauwT8D",
  "model": "text-moderation-007",
  "results": [
    {
      "categories": {
        "harassment": false,
        "harassment/threatening": false,
        "hate": false,
        "hate/threatening": false,
        "self-harm": false,
        "self-harm/instructions": false,
        "self-harm/intent": false,
        "sexual": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false,
        "self-harm": false,
        "sexual/minors": false,
        "hate/threatening": false,
        "violence/graphic": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "harassment/threatening": false
      },
      "category_scores": {
        "harassment": 0.013268317095935345,
        "harassment/threatening": 2.7215596887231186e-8,
        "hate": 0.00010861659393412992,
        "hate/threatening": 5.4016899460329615e-12,
        "self-harm": 8.502685666655907e-9,
      

# Youtube download, transcription and summary

Let's transcribe and ask to summarise (will take a while)

In [ ]:
from pytube import YouTube

(
    chatbot.run_function(
        YouTube("https://www.youtube.com/watch?v=iMo5-4h_XwE")
        .streams.get_audio_only()
        .download(filename="generated_output/zirp.mp3")
    )
    .generate_transcription(filename="generated_output/zirp.mp3", label="transcript")
    .set_model("gpt-3.5-turbo")
    .completion("Summarise the following text for me:\n[transcript]", label="summary_transcript")
)

# The end